In [1]:
from sqlalchemy.orm import declarative_base

# Model

In [2]:
import os
from pathlib import Path
#BASE_DIR = os.path.dirname(os.path.realpath(__file__))
BASE_DIR = Path().resolve() #esto funciona en notebook
connection_string = "sqlite:///" + os.path.join(BASE_DIR, 'site.db')
print (connection_string)

sqlite:////Users/franciscoerrandonea/Code/tut_bbdd/site.db


In [3]:
from sqlalchemy import Column, Integer, String, DateTime, create_engine
from datetime import datetime

Base = declarative_base()

engine = create_engine(connection_string, echo=True)
"""
class User
    id int
    username str
    email str
    date_create datetime
"""

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    username = Column(String(25), nullable=False, unique=True)
    email = Column(String(80), nullable=False, unique=True)
    date_created = Column(DateTime, default=datetime.utcnow())
    
    def __repr__ (self):
        return f"<User username={self.username} email={self.email}>"
    
new_user = User(id=1, username="jonathan", email = "jone@hui.com")
print (new_user)

<User username=jonathan email=jone@hui.com>


In [4]:
#create database if not exists
Base.metadata.create_all(engine)

2022-11-25 14:49:33,182 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-25 14:49:33,183 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-11-25 14:49:33,183 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-25 14:49:33,184 INFO sqlalchemy.engine.Engine COMMIT


session sirve para hacer las transacciones (lazy¿?)

In [5]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker()
local_session = Session(bind=engine)

# Insert

In [6]:
new_user = User(username="frank", email = "frank@sinatra.com")
local_session.add(new_user)
local_session.commit()

2022-11-25 14:49:33,280 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-25 14:49:33,283 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-25 14:49:33,284 INFO sqlalchemy.engine.Engine [generated in 0.00051s] ('frank', 'frank@sinatra.com', '2022-11-25 17:49:33.152605')
2022-11-25 14:49:33,285 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
users = [
    {"username":"jhonny",
    "email":"jh@company.com"},
    {"username":"jerry",
    "email":"je@company.com"},
    {"username":"lucas",
    "email":"lu@company.com"},
    {"username":"igor",
    "email":"ig@company.com"},
    {"username":"gaby",
    "email":"gb@company.com"},
    {"username":"tom",
    "email":"tm@company.com"},
]

In [9]:
for u in users:
    new_user = User(username = u["username"], email = u["email"])
    local_session.add(new_user)    
    local_session.commit()

2022-11-25 14:54:28,052 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-25 14:54:28,054 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-25 14:54:28,054 INFO sqlalchemy.engine.Engine [cached since 294.8s ago] ('jhonny', 'jh@company.com', '2022-11-25 17:49:33.152605')
2022-11-25 14:54:28,056 INFO sqlalchemy.engine.Engine COMMIT
2022-11-25 14:54:28,058 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-25 14:54:28,058 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-25 14:54:28,058 INFO sqlalchemy.engine.Engine [cached since 294.8s ago] ('jerry', 'je@company.com', '2022-11-25 17:49:33.152605')
2022-11-25 14:54:28,059 INFO sqlalchemy.engine.Engine COMMIT
2022-11-25 14:54:28,061 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-25 14:54:28,061 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-11-25 14:54:28,061 I

# Query

In [10]:
users = local_session.query(User).all()
for user in users:
    print (user.username)

2022-11-25 15:10:24,116 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-25 15:10:24,119 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users
2022-11-25 15:10:24,120 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ()
frank
jhonny
jerry
lucas
igor
gaby
tom


In [11]:
users = local_session.query(User).all()[:3]
for user in users:
    print (user.username)

2022-11-25 15:11:02,675 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users
2022-11-25 15:11:02,676 INFO sqlalchemy.engine.Engine [cached since 38.56s ago] ()
frank
jhonny
jerry


# Update

In [12]:
#gaby
user = local_session.query(User).filter(User.username == "gaby").first()


2022-11-25 15:16:06,334 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users 
WHERE users.username = ?
 LIMIT ? OFFSET ?
2022-11-25 15:16:06,338 INFO sqlalchemy.engine.Engine [generated in 0.00304s] ('gaby', 1, 0)


In [13]:
print (user)

<User username=gaby email=gb@company.com>


In [14]:
#updating

In [15]:
user

<User username=gaby email=gb@company.com>

In [16]:
user.username="gabriela"
user.email = "gabriela@company.com"

In [17]:
local_session.commit()

2022-11-25 15:26:05,780 INFO sqlalchemy.engine.Engine UPDATE users SET username=?, email=? WHERE users.id = ?
2022-11-25 15:26:05,781 INFO sqlalchemy.engine.Engine [generated in 0.00123s] ('gabriela', 'gabriela@company.com', 6)
2022-11-25 15:26:05,782 INFO sqlalchemy.engine.Engine COMMIT


# Delete

In [18]:
user_to_delete = local_session.query(User).filter(User.username == "gabriela").first()

2022-11-25 15:27:15,453 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-25 15:27:15,455 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users 
WHERE users.username = ?
 LIMIT ? OFFSET ?
2022-11-25 15:27:15,456 INFO sqlalchemy.engine.Engine [cached since 669.2s ago] ('gabriela', 1, 0)


In [19]:
local_session.delete(user_to_delete)

In [20]:
local_session.commit()

2022-11-25 15:27:32,778 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2022-11-25 15:27:32,778 INFO sqlalchemy.engine.Engine [generated in 0.00076s] (6,)
2022-11-25 15:27:32,780 INFO sqlalchemy.engine.Engine COMMIT


# Order

In [23]:
users = local_session.query(User).order_by(User.username).all()

2022-11-25 15:30:26,057 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-25 15:30:26,059 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users ORDER BY users.username
2022-11-25 15:30:26,060 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ()


In [24]:
users

[<User username=frank email=frank@sinatra.com>,
 <User username=igor email=ig@company.com>,
 <User username=jerry email=je@company.com>,
 <User username=jhonny email=jh@company.com>,
 <User username=lucas email=lu@company.com>,
 <User username=tom email=tm@company.com>]

In [27]:
from sqlalchemy import desc
users = local_session.query(User).order_by(desc(User.username)).all()
users

2022-11-25 15:32:16,842 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users ORDER BY users.username DESC
2022-11-25 15:32:16,848 INFO sqlalchemy.engine.Engine [generated in 0.00624s] ()


[<User username=tom email=tm@company.com>,
 <User username=lucas email=lu@company.com>,
 <User username=jhonny email=jh@company.com>,
 <User username=jerry email=je@company.com>,
 <User username=igor email=ig@company.com>,
 <User username=frank email=frank@sinatra.com>]

In [28]:
import sqlalchemy
sqlalchemy.__version__

'1.4.43'

In [29]:
%reset

# 2.0

In [31]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///some.db", future=True)

In [33]:
type(engine)

sqlalchemy.future.engine.Engine

Es conexión lazy, no está conectado de verdad

In [34]:
connection = engine.connect()
print (connection)

In [35]:
connection.connection.connection